# Load and transform TwoPattern dataset
http://timeseriesclassification.com/description.php?Dataset=TwoPatterns

## Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from time import time
from tqdm import tqdm

time_index = int(time())

DATA_DIR = "data_two_pattern"
DIFFERENT_LENS = True
UNBALANCED = False # if set to unbalanced, samples are shuffled so that data is unbalanced 

NO_TEST_SAMPLES = 800 # there are total 5000 samples
NO_TRAIN_SAMPLES = 200
if NO_TEST_SAMPLES + NO_TRAIN_SAMPLES > 5000:
    raise Exception

infix = "dif" if DIFFERENT_LENS else "eq"
OUTPUT_FILENAME = f"two_pattern_{infix}_lens_ntrain{NO_TRAIN_SAMPLES}.pkl"

## Prepare and save dataset

In [2]:
converted_lines = []
for filename in ["test.txt", "train.txt"]:
    with open (f"{DATA_DIR}/{filename}", "r") as f:
        lines = f.readlines()
        for l in lines:
            floats = list(map(float, l.split()))
            converted_lines.append(floats)

sample_ids = list(range(len(converted_lines)))
true_labels = [int(X[0])-1 for X in converted_lines] # labels should start from 0
all_X_samples = [np.array(X[1:]).reshape(-1,1) for X in converted_lines]

# if different lenghts, cut some of samples
if DIFFERENT_LENS:
    new_samples = []
    for X in all_X_samples:
        cut = np.random.randint(1,40)
        new = X[:-cut]
        new_samples.append(new)
        if len(new) == 0:
            raise Exception
    all_X_samples = new_samples # overwrite sample list

sample_lengths = [sample.shape[0] for sample in all_X_samples]

labels_df = pd.DataFrame({
    "sample_id": sample_ids,
    "true_label": true_labels,
    "sample_len": sample_lengths
})

In [17]:
test_ids = []
train_ids = []

if not UNBALANCED:
    for label, df in labels_df.groupby("true_label"):
        label_train = df.sample(NO_TRAIN_SAMPLES//4).index.values
        label_test = df.drop(label_train).sample(NO_TEST_SAMPLES//4).index.values
        if not set(label_train).isdisjoint(set(label_test)):
            display(df.loc[label_train])
            display(df.loc[label_test])
            print([l for l in label_test if l in label_train])
            raise Exception
        train_ids.append(label_train)
        test_ids.append(label_test)

# make dataset unbalanced -----------
else:
    label_fractions = [0.1, 0.2, 0.2, 0.5]
    for label, df in labels_df.groupby("true_label"):
        label_train = df.sample(label_fractions[label]*NO_TRAIN_SAMPLES//4).index.values
        label_test = df.drop(label_train).sample(label_fractions[label]*NO_TEST_SAMPLES//4).index.values
        if not set(label_train).isdisjoint(set(label_test)):
            display(df.loc[label_train])
            display(df.loc[label_test])
            print([l for l in label_test if l in label_train])
            raise Exception
        train_ids.append(label_train)
        test_ids.append(label_test)
    
indices_splits_lst = [{"test_samples_ids": test_ids, "train_samples_ids": train_ids}]

In [ ]:
output_path = f"{DATA_DIR}/{OUTPUT_FILENAME}_{time_index}"

metadata = {
    "N_TRAIN_SAMPLES_PER_MODEL": len(train_ids),
    "N_TEST_SAMPLES_PER_MODEL": len(test_ids),
    "MAX_SAMPLE_LEN": max(sample_lengths),
    "MIN_SAMPLE_LEN": min(sample_lengths)
}

data = {
    'generating_model': "SIM_TwoPat",
    'data_filename': output_path,
    'time_index': time_index,
    'models_lst': None,
    'labels_df': labels_df,
    'all_X_samples': all_X_samples,
    'indices_splits_lst': indices_splits_lst,
    'metadata': metadata,
    'description': "http://timeseriesclassification.com/description.php?Dataset=TwoPatterns"
}

with open(output_path, 'wb') as f:
    pickle.dump(data, f)
    print(f"Data saved to {output_path}.")

## EDA

### Is dataset balanced?

In [ ]:
# test
labels_df.set_index("sample_id").iloc[test_ids].groupby("true_label").count()

In [ ]:
labels_df.set_index("sample_id").iloc[train_ids].groupby("true_label").count()

Sizes of all classes are roughly equal both in train and test subsets.

### Plots of samples

In [ ]:
labels = ["DD", "UD", "DU", "UU"]
plt.rcParams["figure.figsize"] = (40, 16)
plt.rcParams['font.size'] = 30
fig, axes = plt.subplots(2,2, sharey="all")
plt.suptitle(f"Two samples of each class")
for i, df in tqdm(labels_df.groupby("true_label"), desc = "Preparing plots"):
    ax = axes.flat[i]
    ids = df["sample_id"].sample(2).values
    samples = np.array([all_X_samples[id] for id in ids])
    ax.plot(samples.reshape((2, -1)).T, lw = 3)
    ax.set_title(f"({labels[i]}) label = {i}")
    ax.grid()
    
plt.savefig(f"./plots/models_twoPat_{time_index}.jpg")
plt.show()